In [10]:
# Websites:
# 1. https://www.rent.com/california/los-angeles-apartments/axis-4-100064358
# Scrape data of each available unit.

In [11]:
# Key Data Features:
# - Address and unit number
# - Sq ft
# - Type (studio, home, apartment)
# - Beds
# - Baths
# - Date available
# - Asking rent

In [12]:
! pip install beautifulsoup4

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
! pip install selenium

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from random import randint
pd.set_option('display.max_rows', 500)
import numpy as np


In [15]:
# Install homebrew on mac
# Follow Instructions from https://medium.com/dropout-analytics/selenium-and-geckodriver-on-mac-b411dbfe61bc
# Install geckodriver via homebrew
# Copy Paste "/usr/local/bin/geckodriver" file path. This should remain the same, but in case it does not you 
# you will need to replace every executable_path variable

In [16]:
#confirm geckodriver is in list below
! ls

LinkedIn Crawler_3262020.ipynb        rent_com_get_apt_links_for_city.ipynb
geckodriver.log                       webdriver
rent_com_get_apartment_details.ipynb


In [18]:
import time
start_time = time.time()

options = webdriver.FirefoxOptions()
options.add_argument("start-maximized")
options.add_argument('disable-infobars')

browser = webdriver.Firefox(options=options, 
                           executable_path='/usr/local/bin/geckodriver')

city = 'los-angeles-apartments'.replace('-','_')

browser.get('https://www.rent.com/california/los-angeles-apartments')
result=BeautifulSoup(browser.page_source)

#getting the first page of apartment link results and saving to apartment_link dataframe variable
apartment_link = pd.DataFrame()
for link in result.findAll('a', {'class': '_1Fv8S _3DZdx'}):
    try:
        apartment_link = apartment_link.append(["https://www.rent.com/" + link['href']])
    except KeyError:
        pass

#finding the max amount of pages the geo/dma/msa has
properties_found = result.findAll('div', {'class': '_1Uvv-'})[0]
properties_found_text = properties_found.text
total_properties = int(re.findall(r'\d+', properties_found_text)[0])
max_pages = round(total_properties / len(apartment_link),0)

#Saving next page links to next_page_directory dataframe variable
next_page_directory = pd.DataFrame()
for page_number in range(2, int(max_pages)):
    next_page_directory = next_page_directory.append(['https://www.rent.com/california/los-angeles-apartments?page=%s' %page_number])
next_page_directory.columns = ['page_url']



/Users/bryceeb/anaconda/envs/python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/bryceeb/anaconda/envs/python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
for i, row in next_page_directory.iterrows():
    browser = webdriver.Firefox(options=options, 
                           executable_path='/usr/local/bin/geckodriver')

    browser.get(row['page_url'])
    result=BeautifulSoup(browser.page_source)

    #getting the first page of apartment link results and saving to apartment_link dataframe variable

    for link in result.findAll('a', {'class': '_1Fv8S _3DZdx'}):
        try:
            apartment_link = apartment_link.append(["https://www.rent.com/" + link['href']])
        except KeyError:
            pass
        
    browser.quit()

/Users/bryceeb/anaconda/envs/python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/bryceeb/anaconda/envs/python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
apartment_link = apartment_link.drop_duplicates(keep='first')

In [ ]:
city = 'los-angeles-apartments'.replace('-','_')

In [ ]:
apartment_link.to_csv('%s_urls.csv' % city)